In [31]:
import os
import sys

from typing import Tuple, List, Optional
from qiskit import QuantumCircuit

script_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(script_dir)

from scripts.agents.greedy_agent import GreedyMergeOptimizer


GRIDSYNTH = "gridsynth"
SOLOVAY_KITAEV = "solovay_kitaev"

KAK = "kak"
BQSKit = "bqskit"


In [32]:
from qiskit.circuit.random import random_circuit

my_circuit = random_circuit(num_qubits=5, depth=12, max_operands=2, seed=42)
circuit_name = "my_random_circuit"
my_circuit.draw(fold=-1)

┌─────────────────────────┐                  ┌───┐      ┌────────────┐            ┌───┐                                                                         ┌───────────────────┐         ┌───┐      ┌────────────────────────────┐       ┌────────────┐                                                                                                                    
q_0: ┤0                        ├──────────────────┤ H ├──────┤ Rx(4.6795) ├────────────┤ H ├──────────────────────────────────────────────■───────────────────■──────┤ U2(4.1589,3.4999) ├─────────┤ I ├──────┤ U3(0.55072,0.74145,6.0438) ├───────┤ Rz(4.7659) ├────────────────────────────────────────────────────────────────────────■───────────────────────────────■───────────
     │                         │┌────────────┐    └─┬─┘      └───┬───┬────┘            ├───┴┐          ┌───┐   ┌──────┐                   │                 ┌─┴──┐   └───┬────────────┬──┘    ┌────┴───┴─────┐└────────────────────────────┘       └────────────┘                     ┌──────────────────────────┐                       │             ┌───────────┐     │           
q_1: ┤                         ├┤ Rz(2.3298) ├──────┼────────────┤ X ├─────────────────┤ √X ├──────────┤ Y ├───┤ √Xdg ├───────────────────┼─────────────────┤ Sx ├───────┤ Ry(1.8447) ├───────┤1             ├────────────────────────────────────────────────────────────────■───────┤ U3(2.8637,1.2715,1.9224) ├──────────────■────────┼─────────────┤ P(6.0228) ├─────┼───────────
     │  (XX-YY)(4.939,0.80496) │└────────────┘      │       ┌────┴───┴─────┐┌──────────┴────┴─────────┐├───┤   └──────┘                   │                 ├───┬┘       └───┬────┬───┘       │              │        ┌───────────┐                ┌────────────┐             │       └────────────┬─────────────┘┌─────┐┌──────┴──────┐ │          ┌──┴───────────┴───┐ │           
q_2: ┤                         ├─■──────────────────■───────┤1             ├┤ U(1.4257,4.2086,2.7467) ├┤ H ├──────────────────────────────┼─────────────────┤ H ├────────────┤ √X ├───────────┤  Ryy(2.9021) ├────────┤ P(1.0133) ├────────────────┤ U1(5.7088) ├─────────────┼────────────────────┼──────────────┤ Sdg ├┤ Ry(0.26147) ├─┼──────────┤ R(2.1794,3.7128) ├─┼───────────
     │                         │ │            ┌────────────┐│              │└──────────┬───┬──────────┘├───┤┌────────────┐                │                 └─┬─┘            └────┘           │              │        └───────────┘                └────────────┘       ┌─────┴──────┐             │              └─────┘└─────────────┘ │P(3.1038) └──────────────────┘ │P(0.14328) 
q_3: ┤1                        ├─┼────────────┤ Ry(2.2276) ├┤  Rxx(4.2917) ├───────────┤ Y ├───────────┤ Y ├┤ Rx(2.8835) ├────────────────┼───────────────────■───────────────────────────────┤0             ├──────────────────────────────────────────────────────────┤ Ry(4.3964) ├─────────────■─────────────────────────────────────■───────────────────────────────■───────────
     └─────────────────────────┘ │U1(2.8299)  ├────────────┤│              │       ┌───┴───┴────┐      └───┘└────────────┘┌───────────────┴────────────────┐┌───┐ ┌──────────────────────────┐└┬────────────┬┘           ┌──────┐           ┌──────────────────────────┐└────────────┘                                                                                               
q_4: ────────────────────────────■────────────┤ Ry(4.7632) ├┤0             ├───────┤ Rx(5.2319) ├─────────────────────────┤ U(4.9444,4.1774,4.4307,4.9055) ├┤ Y ├─┤ U3(1.4699,0.36633,1.768) ├─┤ Rz(3.1482) ├────────────┤ √Xdg ├───────────┤ U3(3.6393,1.1107,5.3823) ├─────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                              └────────────┘└──────────────┘       └────────────┘                         └────────────────────────────────┘└───┘ └──────────────────────────┘ └────────────┘            └──────┘           └──────────────────────────┘

In [33]:
from scripts.agents.base_agent import CliffordTSynthesizer

synthesis_config = {
    "EPS": 0.01,
    "DEPTH": 2,
    "MODE1": GRIDSYNTH,
    "MODE2": KAK
}

synthesizer = CliffordTSynthesizer(mode1=synthesis_config["MODE1"],
                                   mode2=synthesis_config["MODE2"],
                                   epsilon=synthesis_config["EPS"],
                                   recursion_degree=synthesis_config["DEPTH"])
synthesized_circ, t_counts = synthesizer(my_circuit)
print(f"Initial T-count: {sum(t_counts)}")

Initial T-count: 614


In [34]:



model_config = {
    "ONLY_INTERACTIONS": True,
}


optimizer = GreedyMergeOptimizer(
    synthesis_config=synthesis_config,
    model_config=model_config,
    target_circuit_and_name=(my_circuit, circuit_name)
    
)

greedy_sequence, greedy_tcount = optimizer.optimize(verbose=True)
print(f"Greedy T-count: {greedy_tcount}")


Target circuit name: my_random_circuit
Number of qubits in target circuit: 5
Preprocessing done. Circuit cleaned.
First synthesis completed in 1.75 seconds.
========== Initial Circuit Statistics ==========
Number of qubits:       5
Number of gates:        43
Initial T-count:        614
Time taken:             1.76 seconds
Possible merges:        8
Merge strategy:         8 / 10 qubit pairs selected (interacting only)


Initial T-count: 614


Optimizing:   3%|▎         | 1/36 [00:00<00:08,  3.95eval/s]

❌ Merge (0, 1) worsens T-count by 6 (new: 620, old: 614)


Optimizing:   6%|▌         | 2/36 [00:00<00:10,  3.33eval/s]

❌ Merge (2, 4) worsens T-count by 20 (new: 634, old: 614)


...

Optimizing:  50%|█████     | 18/36 [00:03<00:02,  6.99eval/s]

❌ Merge (0, 1) worsens T-count by 6 (new: 480, old: 474)
❌ Merge (2, 4) worsens T-count by 20 (new: 494, old: 474)
⚪ Merge (1, 2) yields SAME T-count (474)
❌ Merge (0, 4) worsens T-count by 24 (new: 498, old: 474)


Optimizing:  58%|█████▊    | 21/36 [00:04<00:02,  5.17eval/s]

❌ Merge (2, 3) worsens T-count by 26 (new: 500, old: 474)
⚪ Merge (0, 2) yields SAME T-count (474)
No improving merge found. Terminating.
========== Optimization Report ==========
Applied merges: [(1, 3), (0, 3)]
Final T-count: 474


Final Fidelity: 0.995680
Time taken: 0.11 minutes
Greedy T-count: 474
